In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os as os

In [ ]:
city = "copenhagen"
source_folder = '../csv_data/copenhagen/'
output_folder = "plots/"
disconnect_pairs = "continuous_disconnected_pairs"

subgraph_percentages_tests = [
    [0.6, 0.4],
    [0.25, 0.25, 0.25, 0.25],
    [0.75, 0.25],
    [0.33, 0.34, 0.33],
    [0.85, 0.15],
    [1],
]
# subgraph_percentages_tests = [
#     [0.85, 0.15],
# ]
prune_quantiles = [0.25, 0.5, 0.75, 1]
# prune_quantiles = [0.5]

order = []
for subgraph_percentage in subgraph_percentages_tests:
    order.append('_'.join(map(str, subgraph_percentage)))

discon_points_vs_hist = output_folder+city+"/Disconnected_Points_vs._Histogram_of_Sum_of_Errors with/"
histogram_prunequantile_sum_vs_disconnect = output_folder+"+city+"/histogram_prunequantile_sum_vs_disconnect/"
subgraph_percentage_prunequantile_sum_vs_disconnect = output_folder+city+"/subgraph_percentage_prunequantile_sum_vs_disconnect/"
fixed_pq_and_measure_show_per_percentile = output_folder+city+"/fixed_pq_and_measure_show_per_growth_proportion/"
heatmap = output_folder+city+"/heatmap/"
# create plot folders if dont exist
if not os.path.exists(discon_points_vs_hist):
    os.makedirs(discon_points_vs_hist)
if not os.path.exists(histogram_prunequantile_sum_vs_disconnect):
    os.makedirs(histogram_prunequantile_sum_vs_disconnect)
if not os.path.exists(subgraph_percentage_prunequantile_sum_vs_disconnect):
    os.makedirs(subgraph_percentage_prunequantile_sum_vs_disconnect)
if not os.path.exists(fixed_pq_and_measure_show_per_percentile):
    os.makedirs(fixed_pq_and_measure_show_per_percentile)
if not os.path.exists(heatmap):
    os.makedirs(heatmap)


In [ ]:
measureDict = {}
file_names = []
for file_name in os.listdir(source_folder):
    if file_name.endswith('.csv'):
        file_names.append(file_name)

for file_name in file_names:
    # Split the file name into parts
    parts = file_name.split('_')
    city = parts[0]  # The first part is the city
    measure = parts[1]  # The second part is the measure
        
    # The rest are the percentiles (everything from the third element up to '.csv')
    percentiles = '_'.join(parts[2:]).replace('.csv', '')
    df = pd.read_csv(os.path.join(source_folder, file_name))
    df['percentile'] = percentiles
    df['measure'] = measure  
    if measure not in measureDict:
        measureDict[measure] = [df]
    else:
        measureDict[measure].append(df)

for k,v in measureDict.items():
    print(k)
    print(len(v))



In [ ]:
if city == "copenhagen":
    copenhagen_betweenness_085_015 = pd.read_csv(os.path.join(source_folder, "copenhagen_betweenness_0.85_0.15.csv"))
    pruned_copenhagen_betweenness_085_015 = copenhagen_betweenness_085_015[copenhagen_betweenness_085_015['prune_quantile'] == 0.5]
    #mean of every column in the pruned_amaliesDF
    mean = pruned_copenhagen_betweenness_085_015.mean()
    print(mean)

elif city == "bern":
    bern_betweenness_085_015 = pd.read_csv(os.path.join(source_folder, "bern_betweenness_0.85_0.15.csv"))
    pruned_bern_betweenness_085_015 = bern_betweenness_085_015[bern_betweenness_085_015['prune_quantile'] == 0.5]
    #mean of every column in the pruned_amaliesDF
    mean = pruned_bern_betweenness_085_015.mean()
    print(mean)
    print(bern_betweenness_085_015.head())
elif city == "montreal":
    montreal_betweenness_085_015 = pd.read_csv(os.path.join(source_folder, "montreal_betweenness_0.85_0.15.csv"))
    pruned_montreal_betweenness_085_015 = montreal_betweenness_085_015[montreal_betweenness_085_015['prune_quantile'] == 0.5]
    #mean of every column in the pruned_amaliesDF
    mean = pruned_montreal_betweenness_085_015.mean()
    print(mean)
    print(montreal_betweenness_085_015.head())


In [ ]:
for pq in prune_quantiles:
    percentile = "0.6_0.4"
    a,b,c = pd.concat(measureDict["betweenness"]), pd.concat(measureDict["closeness"]), pd.concat(measureDict["random"])
    # a["measure"], b["measure"], c["measure"] = "betweenness", "closeness", "random"
    megadf = pd.concat([a,b,c])
    megadf = megadf[megadf["prune_quantile"] == pq].copy()
    megadf = megadf[megadf["percentile"] == percentile].copy()
        #only one prunequantile
    megadf['mean_error'] = megadf.groupby('measure')['sum_of_errors'].transform('mean')

    #Create a figure and axes
    fig, ax1 = plt.subplots(figsize=(10, 6))
    # Plot bar chart for mean_error
    # order bars by percentile
    sns.barplot(
        data=megadf,
        x="measure",
        y="mean_error",
        ax=ax1,
        hue="measure",
    )
    ax1.set_ylabel("Mean Error", fontsize=12)
    ax1.set_xlabel("Measure", fontsize=12)
    ax1.tick_params(axis="y", labelsize=10)
    # Add a secondary y-axis for mean_disconnected_pairs
    ax2 = ax1.twinx()
    # Calculate mean_disconnected_pairs for each percentile
    mean_disconnected_cont = (
        megadf.groupby("measure")[disconnect_pairs].mean()
    )
    # Plot points on top of bars
    sns.scatterplot(
        x=mean_disconnected_cont.index,
        y=mean_disconnected_cont.values,
        ax=ax2,
            color="red",
            s=100,
            label="Avg. Continuous Disconnected Pairs",
            # zorder=5,
        )
    mean_disconnected_bikengrowt = (
        megadf.groupby("measure")['bikengrowth_disconnected_pairs'].mean()
    )
    # Plot points on top of bars
    sns.scatterplot(
        x=mean_disconnected_bikengrowt.index,
        y=mean_disconnected_bikengrowt.values,
        ax=ax2,
            color="blue",
            s=100,
            label="Avg. PFF Disconnected Pairs",
            # zorder=5,
        )
    ax2.set_ylabel("Avg. Disconnected Pairs", fontsize=12, color="black")
    ax2.tick_params(axis="y", labelcolor="red", labelsize=10)

        # Improve layout
    ax1.grid(axis="y", linestyle="--", alpha=0.7)

        # Add legend for the scatter points
    ax2.legend(loc="upper right", fontsize=10)
    plt.savefig(subgraph_percentage_prunequantile_sum_vs_disconnect + "_prunequantile:" + str(pq)+ ".png")
    plt.close()

In [ ]:
# add 
def barPlots_percentile_sum_of_errors():
    pq=0.5
    for measure in measureDict:

        mergedPercentileDf = pd.concat(measureDict[measure])
        #only one prunequantile
        mergedPercentileDf = mergedPercentileDf[mergedPercentileDf['prune_quantile'] == pq]
        mergedPercentileDf['mean_error'] = mergedPercentileDf.groupby('percentile')['sum_of_errors'].transform('mean')
        mergedPercentileDf['mean_disconnected_pairs'] = mergedPercentileDf.groupby('percentile')[disconnect_pairs].transform('mean')
        mergedPercentileDf["percentile"] = pd.Categorical(
        mergedPercentileDf["percentile"],
        categories=order,
        ordered=True
    )
        # add line for mean_disconnected_pairs with different y-axis

        sns.catplot(data=mergedPercentileDf, kind="bar", y="mean_error", hue="percentile", errorbar=None).set_axis_labels(measure + " with prune quantile: "+ str(pq), "Mean error")
        sns.catplot(data=mergedPercentileDf, kind="bar", y="mean_disconnected_pairs", hue="percentile", errorbar=None).set_axis_labels(measure + " with prune quantile: "+ str(pq), "Mean disconnected pairs")


In [ ]:
# add 
def barplots_prunequantile_sum_of_errors():
    for measure in measureDict:
        mergedPercentileDf = pd.concat(measureDict[measure])
        mergedPercentileDf['mean_error'] = mergedPercentileDf.groupby('prune_quantile')['sum_of_errors'].transform('mean')
        mergedPercentileDf['mean_disconnected_pairs'] = mergedPercentileDf.groupby('prune_quantile')[disconnect_pairs].transform('mean')
        mergedPercentileDf["prune_quantile"] = pd.Categorical(
        mergedPercentileDf["prune_quantile"],
        categories=prune_quantiles,
        ordered=True
    )
        # add line for mean_disconnected_pairs with different y-axis

        sns.catplot(data=mergedPercentileDf, kind="bar", y="mean_error", hue="prune_quantile", errorbar=None).set_axis_labels(measure, "Mean error")
        sns.catplot(data=mergedPercentileDf, kind="bar", y="mean_disconnected_pairs", hue="prune_quantile", errorbar=None).set_axis_labels(measure, "Mean disconnected pairs")

In [ ]:
def erros_vs_discon_points(df, measure, string_p, prune_quantile=0.5):
    # Bin the data
    bins = 10  # Number of bins
    df["error_bin"] = pd.cut(df["sum_of_errors"], bins=bins)
    
    # Aggregate disconnected pairs per bin
    binned_data = df.groupby("error_bin", observed=True,).agg(avg_disconnected=(disconnect_pairs, "mean"), bin_center=("sum_of_errors", "mean"), bikngrowth=("bikengrowth_disconnected_pairs", "mean")).reset_index()

    # Sort the original data for histogram
    df_sorted = df.sort_values("sum_of_errors")

    # Create the plot
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot the histogram of sum_of_errors on the primary y-axis
    sns.histplot(df_sorted["sum_of_errors"], bins=bins, alpha=0.5, ax=ax1, label="Histogram of Errors")
    ax1.set_xlabel("Sum of Errors")
    ax1.set_ylabel("Frequency of Errors", color="black")
    ax1.tick_params(axis="y", labelcolor="black")

    # Create a secondary y-axis for disconnected points per bin
    ax2 = ax1.twinx()
    ax2.plot(
        binned_data["bin_center"], 
        binned_data["avg_disconnected"], 
        marker="o", 
        label="Avg. Continuous Disconnected Pairs", 
        color="red"
    )
    ax2.set_ylabel("Avg. Disconnected Pairs", color="black")
    ax2.tick_params(axis="y", labelcolor="red")

    ax2.plot(
        binned_data["bin_center"], 
        binned_data["bikngrowth"], 
        marker="o", 
        label="Avg. PFF Disconnected Pairs", 
        color="blue"
    )
    ax2.set_ylim(bottom=-2)
  
    # Add legend
    ax1.legend(loc="upper left", fontsize=10)
    ax2.legend(loc="upper right", fontsize=10)

    plt.title("Disconnected Points vs. Histogram of Sum of Errors with " + str(measure) + " and " + string_p + " Prune Quantile: " + str(prune_quantile)) 
    # Show plot
    plt.tight_layout()
    #plt.show()
    # save the plot
    plt.savefig(discon_points_vs_hist + str(measure) + " and " + string_p + " Prune Quantile: " + str(prune_quantile) + ".png")
    print("plot saved for " + str(measure) + " and " + string_p + " Prune Quantile: " + str(prune_quantile))
    plt.close()


In [ ]:
for measure in measureDict:
    for df in measureDict[measure]:
        percentile = df["percentile"][0]
        for prune_quantile in prune_quantiles:
            #print (measure, percentile, prune_quantile)
            pruned = df[df["prune_quantile"] == prune_quantile].copy()
            erros_vs_discon_points(df=pruned, measure=measure, string_p=percentile, prune_quantile=prune_quantile)

In [ ]:
import math
for measure in measureDict:
    with open(discon_points_vs_hist + "Closest_to_Mean_" + str(measure) + ".txt", "w") as f:
        f.write("")
for measure in measureDict:
    for df in measureDict[measure]:
        percentile = df["percentile"][0]
        for prune_quantile in prune_quantiles:
            pruned = df[df["prune_quantile"] == prune_quantile].copy()
            mean_sum_errors = pruned["sum_of_errors"].mean()
            
            closest = pruned.iloc[(pruned["sum_of_errors"]-mean_sum_errors).abs().argsort()[:1]]
            index=closest.iloc[0].name
            iteration = math.floor(index / len(prune_quantiles))
            closest_sum = closest["sum_of_errors"].values[0]
            with open(discon_points_vs_hist + "Closest_to_Mean_" + str(measure) + ".txt", "a") as f:
                f.write(str(percentile) + ", Prune Quantile: " + str(prune_quantile) + ", index:" + str(index) + ", mean: "+ str(mean_sum_errors) + ", iteration:" + str(iteration) + ", closests sum:" + str(closest_sum) + "\n")
    
        

In [ ]:
from scipy.stats import f_oneway
def correlation_percentile_sum_of_errors_with_prune():
    print("Percentile and sum of errors")
    for measure in measureDict:
        for prune_quantile in prune_quantiles:
            df = pd.concat(measureDict[measure])
            df = df[df["prune_quantile"] == prune_quantile].copy()
            groups = [df[df["percentile"] == level]["sum_of_errors"] for level in df["percentile"].unique()]
            f_stat, p_value = f_oneway(*groups)
            print(f"ANOVA F-statistic for {measure}, prune_quantile {prune_quantile}: {f_stat}, p-value: {p_value}")
    print('\n')
def correlation_percentile_disconnedted_pairs_with_prune():
    print("Percentile and disconnected pairs")
    for measure in measureDict:
        for prune_quantile in prune_quantiles:
            df = pd.concat(measureDict[measure])
            df = df[df["prune_quantile"] == prune_quantile].copy()
            groups = [df[df["percentile"] == level]["continuous_disconnected_pairs"] for level in df["percentile"].unique()]
            f_stat, p_value = f_oneway(*groups)
            print(f"ANOVA F-statistic for {measure}, prune_quantile {prune_quantile}: {f_stat}, p-value: {p_value} with prune_quantile: {prune_quantile}")
    print('\n')
def correlation_percentile_sum_of_errors():
    print("Growth proportions and sum of errors")
    for measure in measureDict:
            df = pd.concat(measureDict[measure])
            groups = [df[df["percentile"] == level]["sum_of_errors"] for level in df["percentile"].unique()]
            f_stat, p_value = f_oneway(*groups)
            print(f"ANOVA F-statistic for {measure}: {f_stat}, p-value: {p_value}")
    print('\n')
def correlation_percentile_disconnedted_pairs():
    print("Growth proportions and disconnected pairs")
    for measure in measureDict:
            df = pd.concat(measureDict[measure])
            groups = [df[df["percentile"] == level]["continuous_disconnected_pairs"] for level in df["percentile"].unique()]
            f_stat, p_value = f_oneway(*groups)
            print(f"ANOVA F-statistic for {measure}: {f_stat}, p-value: {p_value}")
    print('\n')

correlation_percentile_sum_of_errors()
correlation_percentile_disconnedted_pairs()
print('-------------------------------------------------')

In [ ]:
#correlation between sum of errors and disconnected pairs in non linear regression
from scipy.stats import pearsonr
def correlation_sum_of_errors_disconnected_pairs_with_prune():
    for measure in measureDict:
        for prune_quantile in prune_quantiles:
            df = pd.concat(measureDict[measure])
            df = df[df["prune_quantile"] == prune_quantile].copy()
            corr, p_value = pearsonr(df["sum_of_errors"], df["continuous_disconnected_pairs"])
            print(f"Pearson correlation for {measure} sum of erros and disconnedted pairs, prune_quantile {prune_quantile}: {corr}, p-value: {p_value}")
    print('\n')

def correlation_sum_of_errors_disconnected_pairs():
    for measure in measureDict:
            df = pd.concat(measureDict[measure])
            corr, p_value = pearsonr(df["sum_of_errors"], df["continuous_disconnected_pairs"])
            print(f"Pearson correlation for {measure}, sum of erros and disconnedted pairs: {corr}, p-value: {p_value}")
    print('\n')

correlation_sum_of_errors_disconnected_pairs()
print('-------------------------------------------------')

In [ ]:
szell = True

for measure in measureDict:
    for dfM in measureDict[measure]:
        df = dfM.copy()

        percentile = df['percentile'][0]
        df['prune_quantile'] = df['prune_quantile'].astype(str)
        df['mean_error'] = df.groupby('prune_quantile')['sum_of_errors'].transform('mean')


        #Create a figure and axes
        fig, ax1 = plt.subplots(figsize=(10, 6))

        # Plot bar chart for mean_error
        # order bars by percentile

        sns.barplot(
            data=df,
            x="prune_quantile",
            y="mean_error",
            ax=ax1,
            hue="prune_quantile",
            palette="crest",
        )
        ax1.set_ylabel("Mean Error", fontsize=12)
        ax1.set_xlabel("Prune Quantile", fontsize=12)
        ax1.tick_params(axis="y", labelsize=10)

        # Add a secondary y-axis for mean_disconnected_pairs
        ax2 = ax1.twinx()

        # Calculate mean_disconnected_pairs for each percentile
        mean_disconnected_cont = (
            df.groupby("prune_quantile")[disconnect_pairs].mean()
        )
        # Plot points on top of bars
        sns.scatterplot(
            x=mean_disconnected_cont.index,
            y=mean_disconnected_cont.values,
            ax=ax2,
            color="red",
            s=100,
            label="Avg. continous Disconnected Pairs",
            # zorder=5,
        )

        
        if szell:
            mean_disconnected_bikengrowth = (
                df.groupby("prune_quantile")["bikengrowth_disconnected_pairs"].mean()
            )
            sns.scatterplot(
                x=mean_disconnected_bikengrowth.index,
                y=mean_disconnected_bikengrowth.values,
                ax=ax2,
                color="blue",
                s=100,
                label="Avg. PFF Disconnected Pairs",
                # zorder=5,
            )

        ax2.set_ylabel("Avg. Disconnected Pairs", fontsize=12, color="black")
        ax2.tick_params(axis="y", labelcolor="red", labelsize=10)

        # Improve layout
        ax1.grid(axis="y", linestyle="--", alpha=0.5)
        # Add legend for the scatter points
        ax2.legend(loc="upper right", fontsize=10)

        # Show the plot
        # plt.tight_layout()
        plt.savefig(histogram_prunequantile_sum_vs_disconnect + str(measure) + " and "  + " subgraph_size: " + str(percentile) + ".png")
        print("plot saved for " + str(measure) + " and "  + " percentile: " + str(percentile))
        plt.close()


In [ ]:
pq=0.5
for measure in measureDict:
    mergedPercentileDf = pd.concat(measureDict[measure])
    #only one prunequantile
    mergedPercentileDf = mergedPercentileDf[mergedPercentileDf['prune_quantile'] == pq].copy()
    mergedPercentileDf['mean_error'] = mergedPercentileDf.groupby('percentile')['sum_of_errors'].transform('mean')

    mergedPercentileDf["percentile"] = pd.Categorical(
    mergedPercentileDf["percentile"],
    categories=order,
    ordered=True
    )
    #Create a figure and axes
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot bar chart for mean_error
    # order bars by percentile

    sns.barplot(
        data=mergedPercentileDf,
        x="percentile",
        y="mean_error",
        ax=ax1,
        hue="percentile",
    )
    ax1.set_ylabel("Mean Error", fontsize=12)
    ax1.set_xlabel("Growth Proportion", fontsize=12)
    ax1.tick_params(axis="y", labelsize=10)

    # Add a secondary y-axis for mean_disconnected_pairs
    ax2 = ax1.twinx()

    # Calculate mean_disconnected_pairs for each percentile
    mean_disconnected_cont = (
        mergedPercentileDf.groupby("percentile", observed=True)[disconnect_pairs].mean()
    )

    # Plot points on top of bars
    sns.scatterplot(
        x=mean_disconnected_cont.index,
        y=mean_disconnected_cont.values,
        ax=ax2,
        color="red",
        s=100,
        label="Avg. Continuos Disconnected Pairs",
        # zorder=5,
    )

    mean_disconnected_bikengrowth = (
        mergedPercentileDf.groupby("percentile",observed=True)["bikengrowth_disconnected_pairs"].mean()
    )

    sns.scatterplot(
        x=mean_disconnected_bikengrowth.index,
        y=mean_disconnected_bikengrowth.values,
        ax=ax2,
        color="blue",
        s=100,
        label="Avg. PFF Disconnected Pairs",
        # zorder=5,
    )

    ax2.set_ylabel("Avg. Disconnected Pairs", fontsize=12, color="black")
    ax2.tick_params(axis="y", labelcolor="red", labelsize=10)

    # Improve layout
    ax1.grid(axis="y", linestyle="--", alpha=0.7)
    #ax1.set_title(measure+": Mean Error and Disconnected Pairs by Percentile with prune quantile: "+ str(pq) , fontsize=14)

    # Add legend for the scatter points
    ax2.legend(loc="upper right", fontsize=10)

    plt.savefig(fixed_pq_and_measure_show_per_percentile + str(measure) + "prunequantile:"+str(pq)+".png")
    plt.close()

In [ ]:
def heatmapf(df, measure,i):
    corr_matrix = df.corr()
    plt.figure(figsize=(9, 6))
    #make text larger
    sns.set_theme(font_scale=1.2)
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
    print(measure)
    print(i)

    #ensure text is visible
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout(w_pad=1)
    plt.savefig(heatmap + measure + "_" + i + ".png")
    plt.show()
    
    
l = ['0.6_0.4', '0.75_0.25', '0.85_0.15', '0.33_0.34_0.33', '0.25_0.25_0.25_0.25','1']

for measure in measureDict:
    for i in l:
        heatdf = pd.concat(measureDict[measure])
        heatdf = heatdf[heatdf['percentile'] ==i]
        heatdf = heatdf.drop(columns=['bikengrowth_disconnected_pairs','continuous_abstract_edges','bikengrowth_abstract_edges', 'bikengrowth_vertices','percentile','measure', 'continuous_vertices',])
        heatdf["continuous\ndisconnected\npairs"] = heatdf["continuous_disconnected_pairs"]
        heatdf = heatdf.drop(columns=['continuous_disconnected_pairs'])
        heatmapf(heatdf, measure,i)

